In [22]:
from junifer.storage import HDF5FeatureStorage
from julearn.api import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.viz import plot_scores
from julearn.stats.corrected_ttest import corrected_ttest
import pandas as pd
import seaborn as sns
from sklearn.svm import LinearSVC
import warnings


In [23]:
storage = HDF5FeatureStorage(uri='./data/AOMIC_Histograms_Parcels.hdf5')

In [24]:
storage.list_features()

{'a619bd2605f3c1038383188f6c118f98': {'datagrabber': {'class': 'DataladAOMICID1000',
   'native_t1w': False,
   'replacements': ['subject'],
   'patterns': {'BOLD': {'pattern': 'derivatives/fmriprep/{subject}/func/{subject}_task-moviewatching_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
     'space': 'MNI152NLin2009cAsym',
     'mask': {'pattern': 'derivatives/fmriprep/{subject}/func/{subject}_task-moviewatching_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
      'space': 'MNI152NLin2009cAsym'},
     'confounds': {'pattern': 'derivatives/fmriprep/{subject}/func/{subject}_task-moviewatching_desc-confounds_regressors.tsv',
      'format': 'fmriprep'}},
    'T1w': {'pattern': 'derivatives/fmriprep/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz',
     'space': 'MNI152NLin2009cAsym',
     'mask': {'pattern': 'derivatives/fmriprep/{subject}/anat/{subject}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
      'space': 'MNI152NLin2009cAsym'}},
    'V

In [29]:
df_parcellations = storage.read_df('VBM_GM_Schaefer100x17_mean_aggregation')
df_histograms= storage.read_df('VBM_GM_Histogram_100bins_IXI_hist')
df_demographics = pd.read_csv('./data/participants.tsv',sep='\t')
df_demographics.rename(columns={"participant_id": "subject"}, inplace=True)

In [30]:
df_parcellations.columns = df_parcellations.columns.astype(str)
df_histograms.columns = df_histograms.columns.astype(str)

# List of columns for each dataframe
X_parcellations = list(df_parcellations.columns)
X_histograms = list(df_histograms.columns)

# Merge with df_demographics on 'subject'
df_full_parcellations = df_parcellations.merge(df_demographics, on="subject")
df_full_histograms = df_histograms.merge(df_demographics, on="subject")

# Map 'sex' column
#df_full_parcellations['sex'] = df_full_parcellations['sex'].map({'F': 1, 'M': 2})
#df_full_histograms['sex'] = df_full_histograms['sex'].map({'F': 1, 'M': 2})

# Drop rows with NaN values
df_full_parcellations = df_full_parcellations.dropna()
df_full_histograms = df_full_histograms.dropna()



In [31]:
creator = PipelineCreator(problem_type="classification")
creator.add("zscore")
creator.add(
    "svm",
    C=(0.001, 100, "log-uniform"),
)

In [32]:
search_params = {
    "kind": "optuna",
    "cv":5
}

scoring = ["balanced_accuracy", "accuracy"]

scores_hists, model_hists, inspector_hists = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring = scoring,

)


scores_schaefer, model_schaefer, inspector_schaefer = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=5,
    scoring = scoring,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/pipeline/pipeline_creator.py:1041: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the futur

In [33]:
scores_schaefer

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.362427,0.003070,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.823077,1.000000,0.826087,1.000000,92,23,0,0,b10eef89b4192178d482d7a1587a248a
1,0.360552,0.003016,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.871212,0.989362,0.869565,0.989130,92,23,0,1,b10eef89b4192178d482d7a1587a248a
2,0.360982,0.002963,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.846154,0.923913,0.826087,0.923913,92,23,0,2,b10eef89b4192178d482d7a1587a248a
3,0.355429,0.002966,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.643939,1.000000,0.652174,1.000000,92,23,0,3,b10eef89b4192178d482d7a1587a248a
4,0.360550,0.003048,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.837302,1.000000,0.826087,1.000000,92,23,0,4,b10eef89b4192178d482d7a1587a248a


In [34]:
scores_hists

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.367971,0.003209,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.610577,0.732558,0.586207,0.732558,86,29,0,0,bc7087515161a73a5a6aff57863f3803
1,0.365895,0.003033,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.623810,0.733766,0.620690,0.732558,86,29,0,1,bc7087515161a73a5a6aff57863f3803
2,0.360433,0.002999,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.612981,1.000000,0.620690,1.000000,86,29,0,2,bc7087515161a73a5a6aff57863f3803
3,0.363556,0.003118,"OptunaSearchCV(cv=KFold(n_splits=5, random_sta...",0.708333,0.683511,0.714286,0.678161,87,28,0,3,bc7087515161a73a5a6aff57863f3803


In [35]:
scores_hists['model'] = 'AOMIC_Histograms'
scores_schaefer['model'] = 'AOMIC_Schaefer'
plot_scores(scores_schaefer,scores_hists)


BokehModel(combine_events=True, render_bundle={'docs_json': {'2809633c-67bf-447b-964a-b8667fbb5477': {'version…